# Overview
An enviroment to train and evaluate neural networks on rule learning. The language used is stated in the notions of Chomsky.
- Terminal symbols (encoded as integers)
- Nonterminalsymbols and production rules (encoded as lambda-functions)

This is how it works:
1. Section "Libraries and utilities imports libraries and defines useful functions.
2. Section "Data generation" defines functions and a class for generating the training data
3. Section "Networks" defines a function that implements different network architectures.
4. Uses the before defined functions and Derivation class.
  a. "get_training_data()" produces data using the grammar and logic for building derivations in the Derivation class the nonterminals and production rules used by "nonterms()".
  b. A loop using "evaluate()" trains all specified architectures on the data and plots the results.

In [16]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, SimpleRNN, TimeDistributed, LSTM, RepeatVector
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import random as rd
from random import seed, randint, choice, shuffle
import numpy as np
from numpy import argmax
import pandas as pd
from tqdm import tqdm
import copy
import re
import importlib
import utils, calc, generation

In [114]:
importlib.reload(calc)

<module 'calc' from '/home/str/master-s-thesis/calc.py'>

In [10]:
dataset_builder(term_num = int,
                iter_range = list,
                sample_size = int,
                span_arg_num = list)

[1, 6, 2]

In [118]:
calc.app_check(calc.no_e, 0, prem = [[[3],[5]],[5]]) == True

True

In [56]:
calc.symb["NOT"]

[3]